In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


### Libraries

In [9]:
import numpy as np
import itertools
from itertools import product
from scipy.optimize import differential_evolution
from pyswarm import pso
import time

### Input Boolean table

In [10]:
def Inputgen(n):
    salida = list(itertools.product([0, 1], repeat=n))
    return salida

### One state evaluation

In [11]:
def evaluateModel(W,Th,Ig):
  res=Ig
  aux=np.dot(W,Ig)
  aux1=aux-Th
  aux1=aux1.flatten()
  for j in range(0,(len(Ig)-2)):
    if aux1[j]>=0:
      res[j]=1
    else:
      res[j]=0
  return (res)

### N state evaluation

In [12]:
def evaluateModelN11(W,Th,k):
  z=len(W)
  N=(z-2)
  a=np.array(Inputgen(N))
  b=np.ones((2**N,N + 1))
  b[:,:-1] = a
  c=np.ones((2**N,N + 2))
  c[:,:-1] = b
  Ig=c
  output=Ig
  for j in range(0,k):
    for i in range(0, len(Ig)):
      output[i,:]=evaluateModel(W,Th,output[i,:])
  return (output)

In [13]:
def evaluateModelN01(W,Th,k):
  z=len(W)
  N=(z-2)
  a=np.array(Inputgen(N))
  b=np.zeros((2**N,N + 1))
  b[:,:-1] = a
  c=np.ones((2**N,N + 2))
  c[:,:-1] = b
  Ig=c
  #Ig=np.array(Inputgen(z))
  output=Ig
  for j in range(0,k):
    for i in range(0, len(Ig)):
      output[i,:]=evaluateModel(W,Th,output[i,:])
  return (output)

In [14]:
def evaluateModelN00(W,Th,k):
  z=len(W)
  N=(z-2)
  a=np.array(Inputgen(N))
  b=np.zeros((2**N,N + 1))
  b[:,:-1] = a
  c=np.zeros((2**N,N + 2))
  c[:,:-1] = b
  Ig=c
  #Ig=np.array(Inputgen(z))
  output=Ig
  for j in range(0,k):
    for i in range(0, len(Ig)):
      output[i,:]=evaluateModel(W,Th,output[i,:])
  return (output)

In [16]:
Ig=np.array(Inputgen(3))
print(Ig)

[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


Generate input for the case: (ωi,ωim) = (1,1) means high levels

In [17]:
N=8
a=np.array(Inputgen(N))
b=np.ones((2**N,N + 1))
b[:,:-1] = a
c=np.ones((2**N,N + 2))
c[:,:-1] = b

Input11=c
print(Input11)

[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


Generate input for the case: (ωi,ωim) = (0,1) means medium levels

In [18]:
N=8
a=np.array(Inputgen(N))
b=np.zeros((2**N,N + 1))
b[:,:-1] = a
c=np.ones((2**N,N + 2))
c[:,:-1] = b

Input01=c
print(Input01)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [1. 1. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 0. 1.]]


Generate input for the case: (ωi,ωim) = (0, 0) describes low levels

In [19]:
N=8
a=np.array(Inputgen(N))
b=np.zeros((2**N,N + 1))
b[:,:-1] = a
c=np.zeros((2**N,N + 2))
c[:,:-1] = b

Input00=c
print(Input00)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 0. 0.]]


### Fitness function

In [20]:
def fitnessNet(chromo):
  F1=np.array([0,0,0,0,1,1,0,0,1,1])
  F2=np.array([0,1,1,1,0,1,0,1,0,1])
  F3=np.array([0,1,1,1,0,1,0,1,0,0])

  #Real vale
  W=chromo[0:100]

  #Discrete value -1,0,1
  #W=np.rint(chromo[0:100])
  
  W=np.reshape(W,(10,10))
  W[-2,:] = 0
  W[-1,:] = 0
  #print(W)
  Theta=chromo[100:110]
  Theta[8]=0
  Theta[9]=0
  #print(Theta)

  #case 1

  #we will consider 16 iterations
  Og=evaluateModelN11(W,Theta,16)


  c1=Og
  n1=len(c1)

  m1=len(np.where((c1==F1).all(axis=1))[0])
  error1= 1-(m1/n1)

  #case 2

  #we will consider 8 iterations
  Og=evaluateModelN01(W,Theta,8)


  c2=Og
  n2=len(c2)

  m2=len(np.where((c2==F2).all(axis=1))[0])
  error2= 1-(m2/n2)

  #case 3

  #we will consider 8 iterations
  Og=evaluateModelN00(W,Theta,8)


  c3=Og
  n3=len(c3)


  m3=len(np.where((c3==F3).all(axis=1))[0])
  error3= 1-(m3/n3)

  error= error1 + error2 + error3

  return (error)

In [21]:
cr=np.random.uniform(-1, 1, size=110)

In [22]:
fitnessNet(cr)

3.0

In [23]:
print(Input11)

[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


Differential evolution (real)

In [29]:
matNumDEV=500
matNetsDEV=np.zeros((matNumDEV,10,10))
matThDEV=np.zeros((matNumDEV,10))
matResDEV=np.zeros((matNumDEV,5))

for i in range(0,matNumDEV):
    start_time = time.time()
    bounds = np.array([[-1,1]]*110)
    result = differential_evolution(fitnessNet, bounds, maxiter=20, popsize=5,recombination=0.3)
    timef=time.time() - start_time
    matResDEV[i,0]=result.fun
    matResDEV[i,1]=result.nfev
    matResDEV[i,2]=result.nit
    matResDEV[i,3]=timef
    W=result.x[0:100]
    #W=np.rint(result.x[0:100])
    W=np.reshape(W,(10,10))
    W[-2,:] = 0
    W[-1,:] = 0
    matNetsDEV[i]=W
    Theta=result.x[100:110]
    Theta[-2:]=0
    matThDEV[i]=Theta
    print(i)
    print(result.fun)

0
0.6171875
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0
10
1.0
11
0.77734375
12
1.0
13
1.0
14
1.0
15
1.0
16
1.0
17
1.0
18
0.8828125
19
0.953125
20
0.5
21
1.0
22
1.0
23
0.98046875
24
0.64453125
25
1.0
26
1.0
27
1.0
28
1.0
29
1.0
30
1.0
31
1.0
32
1.0
33
0.58984375
34
1.0
35
1.0
36
1.0
37
0.83984375
38
1.0
39
1.0
40
1.0
41
1.0
42
1.0
43
0.99609375
44
1.0
45
0.82421875
46
1.0
47
1.0
48
1.0
49
1.0
50
1.0
51
1.0
52
0.3359375
53
1.0
54
1.0
55
0.8828125
56
1.0
57
1.0
58
1.0
59
1.0
60
1.0
61
1.0
62
0.98828125
63
1.0
64
1.0
65
1.0
66
1.0
67
0.98046875
68
1.0
69
1.0
70
1.0
71
1.0
72
1.0
73
0.12890625
74
1.0
75
1.0
76
1.0
77
0.1015625
78
1.0
79
1.0
80
0.97265625
81
1.0
82
1.0
83
1.0
84
0.54296875
85
1.0
86
1.0
87
1.0
88
1.0
89
1.0
90
1.0
91
1.0
92
0.85546875
93
1.0
94
0.9609375
95
0.8203125
96
1.0
97
0.98046875
98
1.0
99
1.0
100
1.0
101
0.03125
102
1.0
103
1.0
104
0.98828125
105
1.0
106
1.0
107
1.0
108
1.0
109
1.0
110
1.0
111
0.64453125
112
1.0
113
1.0
114
1.0
115
1.0
116
1.0
117
1.0
118

In [42]:
matResDEV[:,0]

array([0.6171875 , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.77734375, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.8828125 , 0.953125  ,
       0.5       , 1.        , 1.        , 0.98046875, 0.64453125,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.58984375, 1.        ,
       1.        , 1.        , 0.83984375, 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99609375, 1.        ,
       0.82421875, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.3359375 , 1.        , 1.        ,
       0.8828125 , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.98828125, 1.        , 1.        ,
       1.        , 1.        , 0.98046875, 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.12890625, 1.     

In [ ]:
matThDEV.shape

In [36]:
np.save('pesosDE',matNetsDEV)

In [38]:
np.save('umbralesDE',matThDEV)

In [39]:
np.save('resultadosDE',matResDEV)

In [34]:
wm = np.load('matrices.npy')

In [35]:
wm.shape

(500, 10, 10)

Differential evolution (-1,0,1)

In [19]:
matNumDEV1=500
matNetsDEV1=np.zeros((matNumDEV1,10,10))
matThDEV1=np.zeros((matNumDEV1,10))
matResDEV1=np.zeros((matNumDEV1,5))

for i in range(0,matNumDEV1):
    start_time = time.time()
    bounds = np.array([[-1,1]]*110)
    result = differential_evolution(fitnessNet, bounds, maxiter=20, popsize=5,recombination=0.3)
    timef=time.time() - start_time
    matResDEV1[i,0]=result.fun
    matResDEV1[i,1]=result.nfev
    matResDEV1[i,2]=result.nit
    matResDEV1[i,3]=timef
    W=result.x[0:100]
    W=np.rint(result.x[0:100])
    W=np.reshape(W,(10,10))
    W[-2,:] = 0
    W[-1,:] = 0
    matNetsDEV1[i]=W
    Theta=result.x[100:110]
    Theta[-2:]=0
    matThDEV1[i]=Theta
    print(i)
    print(result.fun)

0
1.0
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0
10
1.0
11
1.0
12
1.0
13
1.0
14
1.0
15
1.0
16
0.8515625
17
1.0
18
1.0
19
1.0
20
0.2734375
21
1.0
22
1.0
23
1.0
24
1.0
25
1.0
26
1.0
27
1.0
28
1.0
29
1.0
30
1.0
31
1.0
32
1.0
33
1.0
34
1.0
35
1.0
36
1.0
37
1.0
38
1.0
39
1.0
40
1.0
41
1.0
42
1.0
43
0.5625
44
0.5
45
1.0
46
1.0
47
1.0
48
1.0
49
1.0
50
1.0
51
1.0
52
1.0
53
1.0
54
0.78515625
55
1.0
56
1.0
57
1.0
58
0.91015625
59
0.5703125
60
1.0
61
1.0
62
1.0
63
1.0
64
0.7421875
65
1.0
66
1.0
67
1.0
68
1.0
69
0.9453125
70
1.0
71
1.0
72
1.0
73
1.0
74
1.0
75
1.0
76
1.0
77
1.0
78
1.0
79
0.0
80
1.0
81
1.0
82
1.0
83
0.328125
84
1.0
85
1.0
86
1.0
87
1.0
88
1.0
89
1.0
90
1.0
91
0.6953125
92
1.0
93
1.0
94
0.49609375
95
1.0
96
1.0
97
1.0
98
1.0
99
0.75
100
1.0
101
0.9140625
102
1.0
103
0.96875
104
1.0
105
1.0
106
1.0
107
1.0
108
1.0
109
1.0
110
1.0
111
1.0
112
1.0
113
1.0
114
1.0
115
1.0
116
1.0
117
1.0
118
1.0
119
1.0
120
1.0
121
1.0
122
1.0
123
1.0
124
1.0
125
1.0
126
1.0
127
1.0
128
1.0
12

In [20]:
np.save('pesosDE1',matNetsDEV1)

In [21]:
np.save('umbralesDE1',matThDEV1)

In [22]:
np.save('resultadosDE1',matResDEV1)

In [23]:
matResDEV1[:,0]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.8515625 , 1.        , 1.        , 1.        ,
       0.2734375 , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.5625    , 0.5       ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.78515625,
       1.        , 1.        , 1.        , 0.91015625, 0.5703125 ,
       1.        , 1.        , 1.        , 1.        , 0.7421875 ,
       1.        , 1.        , 1.        , 1.        , 0.9453125 ,
       1.        , 1.        , 1.        , 1.        , 1.     

PSO real

In [15]:
pip install fst-pso

Note: you may need to restart the kernel to use updated packages.


In [19]:
import miniful

In [20]:
from fstpso import FuzzyPSO

In [42]:
matNumPSO=500
matNetsPSO=np.zeros((matNumPSO,10,10))
matThPSO=np.zeros((matNumPSO,10))
matResPSO=np.zeros((matNumPSO,5))

dims = 110

for i in range(0,matNumPSO):
    start_time = time.time()
    FP = FuzzyPSO()
    FP.set_search_space( [[-1, 1]]*dims )
    FP.set_fitness(fitnessNet)
    try:
        runaux=FP.solve_with_fstpso(max_iter=100)
    except:
        timef=time.time() - start_time
        matResPSO[i,0]=3
        matResPSO[i,3]=timef
        aux=np.zeros(100)
        W=aux[0:100]
        #W=np.rint(result.x[0:100])
        W=np.reshape(W,(10,10))
        W[-2,:] = 0
        W[-1,:] = 0
        matNetsPSO[i]=W
        Theta=np.zeros(10)
        Theta[-2:]=0
        matThPSO[i]=Theta
    else:
        result =  runaux
        timef=time.time() - start_time
        matResPSO[i,0]=result[1]
        matResPSO[i,3]=timef
        aux=np.array(result[0].B)
        W=aux[0:100]
        #W=np.rint(result.x[0:100])
        W=np.reshape(W,(10,10))
        W[-2,:] = 0
        W[-1,:] = 0
        matNetsPSO[i]=W
        Theta=aux[100:110]
        Theta[-2:]=0
        matThPSO[i]=Theta
    
    
    
    
    
    print(i)
    print(matResPSO[i,0])

Fuzzy Self-Tuning PSO - v1.8.1
 * Max distance: 20.976177
 * Search space boundaries set to: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1

In [47]:
np.save('pesosPSO',matNetsPSO)

In [48]:
np.save('umbralesPSO',matThPSO)

In [49]:
np.save('resultadosPSO',matResPSO)

In [43]:
matResPSO[:,0]

array([1.        , 3.        , 1.        , 3.        , 1.640625  ,
       1.        , 1.58984375, 1.        , 1.        , 1.        ,
       1.        , 2.        , 2.        , 2.        , 1.        ,
       3.        , 2.        , 3.        , 1.        , 1.        ,
       2.        , 1.        , 2.        , 1.        , 1.        ,
       3.        , 1.        , 1.5703125 , 3.        , 3.        ,
       2.        , 1.        , 3.        , 1.        , 1.        ,
       1.        , 2.        , 0.        , 1.        , 0.37890625,
       1.265625  , 2.        , 2.        , 3.        , 1.        ,
       2.        , 1.        , 3.        , 3.        , 1.        ,
       0.0546875 , 2.        , 3.        , 3.        , 1.        ,
       1.        , 1.        , 2.        , 1.        , 1.        ,
       2.        , 1.        , 3.        , 1.        , 3.        ,
       3.        , 1.        , 1.        , 3.        , 3.        ,
       2.        , 1.        , 2.        , 3.        , 2.     

In [44]:
matResPSO[:,3]

array([1.39863446e+02, 1.37490797e+00, 1.39172073e+02, 1.37979579e+00,
       1.38764106e+02, 1.39806890e+02, 1.39373552e+02, 1.39714002e+02,
       1.39832035e+02, 1.39835978e+02, 1.40002666e+02, 1.39197811e+02,
       1.39803223e+02, 1.39720806e+02, 1.40492456e+02, 1.37601209e+00,
       1.68311252e+02, 1.37613511e+00, 1.39095943e+02, 1.39748219e+02,
       1.40496544e+02, 1.40608271e+02, 1.39617029e+02, 1.41380762e+02,
       1.39973515e+02, 1.38150024e+00, 1.40351833e+02, 1.40208620e+02,
       1.41429400e+00, 1.40799594e+00, 1.40185708e+02, 1.41333134e+02,
       1.38540888e+00, 1.40618351e+02, 1.40336882e+02, 1.40471579e+02,
       1.40137631e+02, 1.40055913e+02, 1.41197226e+02, 1.41245009e+02,
       1.40348236e+02, 1.40202074e+02, 1.39285504e+02, 1.41932201e+00,
       1.40202370e+02, 1.40869434e+02, 1.40800155e+02, 1.39795399e+00,
       1.37915897e+00, 1.40784773e+02, 1.41590258e+02, 1.40416284e+02,
       1.37767100e+00, 1.37165403e+00, 1.39464099e+02, 1.40370546e+02,
      

In [46]:
matNetsPSO[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [38]:
matThPSO[0]

array([ 0.76928259, -0.35895377, -0.35204918,  0.15244932, -0.16791977,
       -0.05544998,  0.10878961, -0.01321096,  0.        ,  0.        ])

PSO (-1,0,1)

In [51]:
matNumPSO1=500
matNetsPSO1=np.zeros((matNumPSO1,10,10))
matThPSO1=np.zeros((matNumPSO1,10))
matResPSO1=np.zeros((matNumPSO1,5))

dims = 110

for i in range(0,matNumPSO1):
    start_time = time.time()
    FP = FuzzyPSO()
    FP.set_search_space( [[-1, 1]]*dims )
    FP.set_fitness(fitnessNet)
    try:
        runaux=FP.solve_with_fstpso(max_iter=100)
    except:
        timef=time.time() - start_time
        matResPSO1[i,0]=3
        matResPSO1[i,3]=timef
        aux=np.zeros(100)
        #W=aux[0:100]
        W=np.rint(aux[0:100])
        W=np.reshape(W,(10,10))
        W[-2,:] = 0
        W[-1,:] = 0
        matNetsPSO1[i]=W
        Theta=np.zeros(10)
        Theta[-2:]=0
        matThPSO1[i]=Theta
    else:
        result =  runaux
        timef=time.time() - start_time
        matResPSO1[i,0]=result[1]
        matResPSO1[i,3]=timef
        aux=np.array(result[0].B)
        #W=aux[0:100]
        W=np.rint(aux[0:100])
        W=np.reshape(W,(10,10))
        W[-2,:] = 0
        W[-1,:] = 0
        matNetsPSO1[i]=W
        Theta=aux[100:110]
        Theta[-2:]=0
        matThPSO1[i]=Theta
    
    
    
    
    
    print(i)
    print(matResPSO1[i,0])

Fuzzy Self-Tuning PSO - v1.8.1
 * Max distance: 20.976177
 * Search space boundaries set to: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1

In [53]:
np.save('pesosPSO1aux',matNetsPSO1)

In [54]:
np.save('umbralesPSO1aux',matThPSO1)

In [55]:
np.save('resultadosPSO1aux',matResPSO1)

In [56]:
matResPSO1[:,0]

array([3.        , 1.        , 2.        , 3.        , 1.        ,
       1.        , 3.        , 0.        , 3.        , 1.        ,
       1.        , 3.        , 3.        , 1.        , 3.        ,
       3.        , 3.        , 1.        , 2.        , 1.        ,
       1.        , 3.        , 1.        , 1.        , 3.        ,
       1.        , 2.        , 1.        , 0.        , 1.        ,
       3.        , 2.        , 3.        , 2.        , 3.        ,
       1.        , 3.        , 3.        , 3.        , 1.        ,
       2.        , 3.        , 1.        , 3.        , 3.        ,
       3.        , 1.        , 1.        , 1.        , 2.        ,
       3.        , 1.        , 3.        , 1.        , 3.        ,
       1.        , 3.        , 3.        , 3.        , 3.        ,
       1.        , 1.        , 1.        , 3.        , 1.        ,
       1.        , 3.        , 1.        , 3.        , 2.        ,
       3.        , 1.        , 2.        , 2.        , 3.     

In [57]:
matResPSO1[:,3]

array([  1.40830207, 138.15623212, 140.76232982,   1.37725091,
       138.64078307, 140.13132   ,   1.38068008, 139.85729027,
         1.39026093, 139.16478395, 141.06579685,   1.37710786,
         1.39129901, 141.08885288,   1.37577605,   1.37339401,
         1.37421083, 141.28319693, 141.29735279, 140.80443406,
       139.73923993,   1.39788604, 140.46926498, 141.00657988,
         1.37752104, 140.54414678, 140.754987  , 140.73514795,
       140.37200308, 140.83784199,   1.41045666, 139.10975003,
         1.39205384, 141.4813509 ,   1.37372494, 140.35414982,
         1.41604805,   1.39004397,   1.39592099, 139.64254093,
       139.35089803,   1.39060521, 139.69993591,   1.38786888,
         1.39041305,   1.39295006, 143.02168894, 142.06950498,
       141.26382017, 139.79780102,   1.39566278, 140.34075689,
         1.39322972, 140.56607795,   1.40657306, 140.31728888,
         1.38551402,   1.38268018,   1.37145495,   1.38422513,
       139.187958  , 139.91370869, 140.04626203,   1.37

In [60]:
matNetsPSO1[1]

array([[ 0., -1., -1., -0.,  0.,  1., -1.,  0.,  0.,  0.],
       [-0., -0.,  0., -0., -0.,  0.,  0.,  1., -0., -1.],
       [ 0.,  1.,  1., -1.,  0., -0.,  0.,  0., -0., -0.],
       [-1.,  0.,  0.,  0.,  1., -0., -0.,  0., -0., -0.],
       [-0., -1., -0., -0.,  0., -1., -0.,  0., -1., -0.],
       [-0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.],
       [-0., -1., -1., -0., -0., -1.,  0., -0., -0.,  0.],
       [-1.,  0.,  1., -1., -0.,  1.,  0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [61]:
matThPSO1[1]

array([-0.28200312, -0.42133375, -0.24391229, -0.29687148,  0.54508526,
       -0.044097  ,  0.0410031 , -0.03720972,  0.        ,  0.        ])